In [3]:
import numpy as np
import pandas as pd
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm, cross_validation
from sklearn.model_selection import cross_val_score
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor


In [2]:
#required functions

#locating non-numeric values so that I can replace them as NaN value 
def isnumber(x):
    try:
        float(x)
        return True
    except:
        return False

#removing null rows in specfic column(s) from a df
def rm_null(df, i):
    return df[df.iloc[:,i].isnull()]

In [4]:
##I just like to attach column names
hedgecolumns=pd.read_csv("columnnames.csv", encoding =  "ISO-8859-1")
hcol=hedgecolumns.values.T.tolist()[0]

##reading data.csv
hedgeable=pd.read_csv("data_orginal.csv", names=hcol, low_memory=False, skipinitialspace=True)#, skipinitialspace=True, na_values=["?"])
print(len(hedgeable))


3279


In [5]:
#########Dealing with null values#########

#First, I noticed the null values have string that involves "?"
#I replace non-numeric values with NaN in all attribute values
hedgeable.iloc[:,:-1]=hedgeable.iloc[:,:-1][hedgeable.iloc[:,:-1].applymap(isnumber)]

#The instruction stated that "28% of instances are missing some of the continuous attributes."
#However, one binary column also had 15 null values. 15 out of 3,279 is a small number that I wanted to simply omit.
#I simply omit NaN values in the binary attribute columns & response variable column

hedgeable=hedgeable.dropna(subset=hedgeable.columns[3:-1], how='any')
len(hedgeable)


3264

In [6]:
##28% of instances are missing some of the continuous attributes.
##Too big to simply omit in this case. (will try later anyways)
##So, I'm gonna RF regress for the missing data

##hedgeable rows without na values
hedgeclean=hedgeable.dropna()
##hedgeable rows only with na values
hedgena=hedgeable[hedgeable.isnull().any(axis=1)]


#getting rows where each column has null values. So I can replace these with predicted values


height=rm_null(hedgeable, 0)    
width =rm_null(hedgeable, 1)  
aratio=rm_null(hedgeable, 2)  
#print(len(hedgeable[hedgeable.iloc[:,0].isnull()]))
#print(len(hedgeable[hedgeable.iloc[:,1].isnull()]))
#print(len(hedgeable[hedgeable.iloc[:,2].isnull()]))
print(len(height))
print(len(width))
print(len(aratio))

898
896
905


In [7]:
#To fill NaN values of the three continuous variables, I am predicting theses values using random forest
#height and weight could use both RF regressor & Classifier
#The minimum number of groups for any class cannot be less than n_splits=3
#So I am trying regressors: svm and rf 

#print(len(df_width_cl.value_counts())) #219
#print(len(df_width_cl.value_counts())) #277

#predictor
df_x_cl=hedgeclean.iloc[:,3:-1] #2359 rows × 1555 columns

#response
df_height_cl=hedgeclean.iloc[:,0] #2359 rows × 1 columns
df_width_cl=hedgeclean.iloc[:,1] #2359 rows × 1 columns
df_aratio_cl=hedgeclean.iloc[:,2] #2359 rows × 1 columns

#splitting test vs train
x_train_ht, x_test_ht, y_train_ht, y_test_ht = train_test_split(df_x_cl, df_height_cl, test_size=0.2, random_state=1)
x_train_wdt, x_test_wdt, y_train_wdt, y_test_wdt = train_test_split(df_x_cl, df_width_cl, test_size=0.2, random_state=1)
x_train_rt, x_test_rt, y_train_rt, y_test_rt = train_test_split(df_x_cl, df_aratio_cl, test_size=0.2, random_state=1)

In [21]:
#tuning tree counts
#120 had the highest score
#max 330 as my computer can't handle more
#for i in range(30, 330, 30):
#    rf_reg=RandomForestRegressor(n_estimators=i)
#    print("height : ", i, cross_val_score(rf_reg, x_train_ht, y_train_ht).mean())

for i in range(30, 330, 30):
    rf_reg=RandomForestRegressor(n_estimators=i)
    print("width : ",i, cross_val_score(rf_reg, x_train_wdt, y_train_wdt).mean())

for i in range(30, 330, 30):
    rf_reg=RandomForestRegressor(n_estimators=i)
    print("aratio : ",i, cross_val_score(rf_reg, x_train_rt, y_train_rt).mean())

width :  30 0.568004476362
width :  60 0.575271764986
width :  90 0.578605561024
width :  120 0.581520937364
width :  150 0.580798096668
width :  180 0.582325410043
width :  210 0.581150125089
width :  240 0.58365447797
width :  270 0.581455706586
width :  300 0.581801932656
aratio :  30 0.841886360592
aratio :  60 0.842822824785
aratio :  90 0.846051069349
aratio :  120 0.848040482333
aratio :  150 0.847761839956
aratio :  180 0.849426302956
aratio :  210 0.847748582401
aratio :  240 0.850097370525
aratio :  270 0.849766646647
aratio :  300 0.849752789176


In [18]:
#model types
#rf=RandomForestClassifier(n_estimators=50)
rf_reg=RandomForestRegressor(n_estimators=120)
#svc=svm.SVC()
svr=svm.SVR()

#cv
cv = cross_validation.KFold(len(df_x_cl), n_folds=10)

#cv 
regressor_scores = cross_val_score(rf_reg, x_train_ht, y_train_ht)
#svm_scores = cross_val_score(svr, x_train_ht, y_train_ht)
#print("classifier score: ", classifier_scores.mean())
print("rf_regressor_score: ", regressor_scores.mean())
#print("svm_score: ", svm_scores.mean()) 

rf_regressor_score:  0.370916819125


90 0.365810508743
120 0.360952888189
150 0.360552870841
180 0.364811135185
210 0.372527306871
240 0.371034587614


In [12]:
#applying fit model to the test set
rf_reg.fit(x_test_ht, y_test_ht)
svr.fit(x_test_ht,y_test_ht)

#preditcted values
rf_pred_ht_rf=rf_reg.predict(x_test_ht)
rf_pred_ht_svr=svr.predict(x_test_ht)
s=y_test_ht.tolist()

rf_squarederror=[]
sv_squarederror=[]
for i in range(len(rf_pred_ht_rf)):
    rf_squarederror.append(math.sqrt(((rf_pred_ht_rf[i]-float(s[i]))**2)))
    sv_squarederror.append(math.sqrt(((rf_pred_ht_svr[i]-float(s[i]))**2)))


print(sum(rf_squarederror)/len(rf_squarederror))
print(np.mean(rf_squarederror))
print(np.std(rf_squarederror))

#pd.Series(list(s))
print(sum(sv_squarederror)/len(sv_squarederror))
print(np.mean(sv_squarederror))
print(np.std(sv_squarederror))

12.099236377188383
12.0992363772
20.4318422734
39.65887586289224
39.6588758629
47.7784336646


In [116]:
#given the score above, I am going to use the randomforest regressor for predicting the missing values
height_x=height.iloc[:,3:-1]
width_x=width.iloc[:,3:-1]
aratio_x=aratio.iloc[:,3:-1]

rf_reg.fit(df_x_cl, df_height_cl)

height_pred=rf_reg.predict(height_x)
#width_pred=rf_reg.predict(width_x)
#aratio_pred=rf_reg.predict(aratio_x)

In [100]:
print(len(height_pred))
print(len(y_test_ht.values))

898
472


12.044862087136268
12.0448620871
19.6251025719
0.0


In [99]:
#replacing NaN values with the predicted values
hedgeable.loc[hedgeable['height: continuous.'].isnull(), 'height: continuous.']=list(height_pred)
hedgeable.loc[hedgeable['width: continuous.'].isnull(), 'width: continuous.']=list(width_pred)
hedgeable.loc[hedgeable['aratio: continuous.'].isnull(), 'aratio: continuous.']=list(aratio_pred)


In [100]:
#making sure there's no null values
print(len(hedgeable))
print(len(hedgeable.dropna()))

3264
3264


In [122]:
x=hedgeable.iloc[:,:-1]
ad=hedgeable.iloc[:,-1]
x_train, x_test, y_train, y_test = train_test_split(x, ad, test_size=0.3)#, random_state=3)

#classifying now 
rf=RandomForestClassifier(n_estimators=100)
rf.fit(x_train, y_train)
pred=rf.predict(x_test)


s=y_test.values

squarederror=[]
correct=0
for i in range(len(pred)):
    if pred[i]==s[i]:
        correct=correct+1
    else:
        correct=correct+0

print(correct/len(s))

0.976530612244898


In [118]:
s=y_test.values

squarederror=[]
correct=0
for i in range(len(pred)):
    if pred[i]==s[i]:
        correct=correct+1
    else:
        correct=correct+0

print(correct/len(s))

0.9806122448979592
